# Provided Models

## Vanilla LDA

Plain 'ol LDA

Only works for classic 96-dimensional mutational signatures 

## Tandem LDA

Two layers of LDA
Infers activities for damage and repair signatures

Can be initialised with classic 96-dimensional mutational signatures 

## Damuta 

The full damuta model is similar to the Tandem LDA model, with an added hirearchical proir to incorporate tissue-specific information. 

Infers activities for damage and repair signatures

Can be initialised with classic 96-dimensional mutational signatures 